In [3]:
from research_copilot.utils import count_tokens_in_file
cost_per_token = 0.05 / 1000 # input: $0.03 / 1K tokens	output: $0.06 / 1K tokens
tokens = count_tokens_in_file('../data/ccpa.txt')
print(tokens)
print(cost_per_token*tokens)

32022
1.6011


In [34]:
%load_ext autoreload
%autoreload all

INFO:sentence_transformers.SentenceTransformer:Load pretrained SentenceTransformer: thenlper/gte-large
INFO:sentence_transformers.SentenceTransformer:Use pytorch device: cuda


The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [28]:
import logging
logging.basicConfig(level=logging.INFO)

In [31]:
from research_copilot.agent import PrivacyPolicyAgent
agent = PrivacyPolicyAgent()
agent.scan_legislation()

INFO:research_copilot.agent:Running PrivacyPolicyAgent
INFO:research_copilot.agent:Scanning legislation for new requirements
INFO:research_copilot.agent:Scanning legislation for new requirements
Batches: 100%|██████████| 1/1 [00:00<00:00, 74.27it/s]
INFO:openai:error_code=None error_message='Request failed due to server shutdown' error_param=None error_type=server_error message='OpenAI API error received' stream_error=False
  "error": {
    "message": "Request failed due to server shutdown",
    "type": "server_error",
    "param": null,
    "code": null
  }
}
 500 {'error': {'message': 'Request failed due to server shutdown', 'type': 'server_error', 'param': None, 'code': None}} {'Date': 'Sat, 07 Oct 2023 08:15:16 GMT', 'Content-Type': 'application/json', 'Content-Length': '141', 'Connection': 'keep-alive', 'access-control-allow-origin': '*', 'openai-model': 'gpt-4-0613', 'openai-organization': 'user-krxvh9fxrkwdycyyhq58r7vl', 'openai-processing-ms': '15056', 'openai-version': '2020-1

In [35]:
from research_copilot.data_loading.privacy_policy import load_privacy_policy
load_privacy_policy("https://recroom.com/privacy-policy")

INFO:unstructured:Reading document from string ...
INFO:unstructured:Reading document ...


In [51]:
agent.scan_policy()

In [56]:
from research_copilot.agent import PrivacyPolicyAgent
agent = PrivacyPolicyAgent()
agent.check_for_fullfilled_requirements()

ClientError: {code: Neo.ClientError.Procedure.ProcedureCallFailed} {message: Failed to invoke procedure `db.index.vector.queryNodes`: Caused by: java.lang.IllegalArgumentException: There is no such vector schema index: hypothetical-requirement-embedding-index}

sleepytime!

todo
- optimize get_all_codepiece_requirements, takes too long
- fix hypothetical_requirement_similarity_search
- verify mark_fullfilled_by
- verify mark_not_fullfilled_by

- resolve list of unfulfilled requirements and their codepieces

- suggest changes

In [1]:
from research_copilot.data_loading.ca_code import create_codepiece_requirements_index
create_codepiece_requirements_index()

In [1]:
from research_copilot.data_loading.ca_code import get_unfulfilled_codepiece_requirements
reqs = get_unfulfilled_codepiece_requirements()

/home/taylor/Projects/ofcounsel/research-copilot/.venv/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
list(reqs[0]['node'].keys())

['gte-large_path',
 'section_num',
 'is_requirement',
 'requirement_summary',
 'part',
 'title',
 'division',
 'gte-large',
 'path_text',
 'id',
 'text',
 'p_id',
 'requirement_vector',
 'law_code']

In [3]:
reqs[0]['node']['is_requirement']

True

In [9]:
from research_copilot.db.privacy_policy import create_hypothetical_requirement_embedding_index
create_hypothetical_requirement_embedding_index()

In [7]:
# START
from research_copilot.db.privacy_policy import get_all_privacy_policy_sections
secs = get_all_privacy_policy_sections()

In [8]:
from sentence_transformers import SentenceTransformer
embedding_model = SentenceTransformer('thenlper/gte-large')
embedding = embedding_model.encode([secs[0]['node']['text']])
matches = hypothetical_requirement_similarity_search(embedding[0])
print(secs[0]['node']['text'])
print(matches)

INFO:sentence_transformers.SentenceTransformer:Load pretrained SentenceTransformer: thenlper/gte-large
INFO:sentence_transformers.SentenceTransformer:Use pytorch device: cuda
Batches: 100%|██████████| 1/1 [00:00<00:00,  4.63it/s]

Terms of Service
[{'node': {'id': 'df1bb85873a7c434c660256e454421e5-hreq-3', 'text': '6.4. Service Providers: The Company may employ third party companies and individuals to facilitate our Service, to provide the Service on our behalf, or to perform Service-related services. These third parties have access to your Personal Information only to perform these tasks on our behalf and are obligated not to disclose or use it for any other purpose.', 'gte-large': [0.0007706110482104123, -0.022079026326537132, -0.028394214808940887, -0.0048646521754562855, 0.015787992626428604, 0.01682344079017639, -0.014038299210369587, 0.03031439520418644, 0.026315324008464813, 0.033832963556051254, 0.02016456425189972, 0.009659677743911743, 0.023664144799113274, -0.0211341492831707, -0.010576910339295864, 0.0004071724251843989, -0.02622191421687603, 0.002434246474877, -0.004331829026341438, -0.019010217860341072, 0.009517786093056202, 0.015253686346113682, -0.04498966410756111, -0.016437074169516563, -0.030

In [9]:
from research_copilot.db.privacy_policy import get_all_hypothetical_requirements
h_reqs = get_all_hypothetical_requirements()

In [10]:
list(h_reqs[0].keys())

['node', 'hypothetical_requirements']

In [11]:
from research_copilot.data_loading.ca_code import codepiece_text_similarity_search, codepiece_fulltext_search
# text = secs[0]['node']['text']
text = h_reqs[0]['hypothetical_requirements']['text']
cp_matches = codepiece_text_similarity_search(text, embedding_model, embedding_name='gte-large_path')
# pt_matches = codepiece_fulltext_search(text) 
print(text)
print(cp_matches[0]['node']['path_text'])
# print(pt_matches)

Batches: 100%|██████████| 1/1 [00:00<00:00, 149.84it/s]

Requirement 2: The business must include a clause in the terms of service that outlines the conditions under which the business may terminate a user's access to the service. This is necessary to comply with digital rights management regulations that require businesses to specify the grounds for termination.
Methods of Limiting Sale, Sharing, and Use of Personal Information and Use of Sensitive Personal Information | (1)A business shall not be required to comply with subdivision (a) if the business allows consumers to opt out of the sale or sharing of their personal information and to limit the use of their sensitive personal information through an opt-out preference signal sent with the consumer’s consent by a platform, technology, or mechanism, based on technical specifications set forth in regulations adopted pursuant to paragraph (20) of subdivision (a) of Section 1798.185, to the business indicating the | A business that complies with subdivision (a) is not required to comply with 

In [17]:
from research_copilot.db.privacy_policy import hypothetical_requirement_similarity_search
hr_matches = hypothetical_requirement_similarity_search(cp_matches[0]['node']['gte-large_path'])
print(len(hr_matches))
print(hr_matches[0]['node']['text'])

10
Requirement 2: The law mandates that businesses can only collect, use, or store Personal Information for legitimate purposes that have been clearly communicated to the individual. The business must not use the information for any other purpose without obtaining further consent.


In [19]:
from research_copilot.data_loading.ca_code import get_unfulfilled_codepiece_requirements
reqs = get_unfulfilled_codepiece_requirements()


In [20]:
ghr_matches = hypothetical_requirement_similarity_search(reqs[0]['node']['requirement_vector'])
print(reqs[0]['node']['requirement_summary'])
print(ghr_matches[0]['node']['text'])

The business is required to include a clear and noticeable link on its internet homepages. This link should be titled "Do Not Sell or Share My Personal Information". This is to ensure that the business complies with regulations limiting the sale, sharing, and use of personal information and sensitive personal information of consumers. The link should be reasonably accessible to consumers.
Requirement 2: The 'Personal Information Protection and Electronic Documents Act' requires businesses to advise users against sharing private information about themselves on the platform to ensure their personal information is not misused.


In [21]:
all_matches = [hypothetical_requirement_similarity_search(req['node']['requirement_vector']) for req in reqs]

In [25]:
non_empty_matches = [matches for matches in all_matches if matches]
count_non_empty_matches = len(non_empty_matches)
print(count_non_empty_matches)
print(len(all_matches))

255
255


In [4]:
from research_copilot.agent import PrivacyPolicyAgent
agent = PrivacyPolicyAgent()
agent.check_for_fullfilled_requirements()

INFO:openai:error_code=rate_limit_exceeded error_message='Rate limit reached for 10KTPM-200RPM in organization org-XNphF0XRL1KUVWkxlwpJemAZ on tokens per min. Limit: 10000 / min. Please try again in 6ms. Contact us through our help center at help.openai.com if you continue to have issues.' error_param=None error_type=tokens message='OpenAI API error received' stream_error=False
INFO:openai:error_code=rate_limit_exceeded error_message='Rate limit reached for 10KTPM-200RPM in organization org-XNphF0XRL1KUVWkxlwpJemAZ on tokens per min. Limit: 10000 / min. Please try again in 6ms. Contact us through our help center at help.openai.com if you continue to have issues.' error_param=None error_type=tokens message='OpenAI API error received' stream_error=False


KeyboardInterrupt: 

In [28]:
list(hr_matches[0]['node'].keys())

['id', 'text', 'gte-large']

In [29]:
list(reqs[0]['node'].keys())

['gte-large_path',
 'section_num',
 'is_requirement',
 'requirement_summary',
 'part',
 'title',
 'division',
 'gte-large',
 'path_text',
 'id',
 'text',
 'p_id',
 'requirement_vector',
 'law_code']

In [ ]:
req: 'CIV1798.135.202013_a_1'
hreq '1b2d1f1a7d5936d73688e65510902c34-hreq-1'